Initialization

In [ ]:
run(```psql -f sqlsnippets/droptables.sql```)

In [ ]:
push!(LOAD_PATH, "app/resources/insurancecontracts")
import Base: @kwdef
import InsuranceContractsController
using Pkg
Pkg.add("BitemporalPostgres")
Pkg.add("HTTP")
Pkg.add("JSON")
Pkg.add("TimeZones")
Pkg.add("ToStruct")
Pkg.add("Intervals")
Pkg.add("SearchLight")
Pkg.add("SearchLightPostgreSQL")
Pkg.add("Test")
using Test
using InsuranceContractsController.InsuranceContracts
using InsuranceContractsController.InsurancePartners
using InsuranceContractsController.InsuranceTariffs
using BitemporalPostgres
using SearchLight
using TimeZones
using ToStruct
using JSON
using HTTP
using Genie, Genie.Router, Genie.Requests
SearchLight.Configuration.load() |> SearchLight.connect
SearchLight.Migrations.create_migrations_table()
BitemporalPostgres.up()
SearchLight.Migrations.up()


Create Contract blue

In [ ]:

# create Partner
p = Partner()
pr = PartnerRevision(description="blue")
w = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w)
create_component!(p, pr, w)
commit_workflow!(w)

# create Partner
p2 = Partner()
p2r = PartnerRevision(description="blue")
w = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w)
create_component!(p2, p2r, w)
commit_workflow!(w)

# create Tariff
t = Tariff()
tr = TariffRevision(description="blue")
w0 = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w0)
create_component!(t, tr, w0)
commit_workflow!(w0)

# create Contract
c = Contract()
cr = ContractRevision(description="blue")
cpr = ContractPartnerRef(ref_super=c.id)
cprr = ContractPartnerRefRevision(ref_partner=p.id, description="blue")

cpi = ProductItem(ref_super=c.id)
cpir = ProductItemRevision(position=1, description="blue")

pitr = ProductItemTariffRef(ref_super=cpi.id)
pitrr = ProductItemTariffRefRevision(ref_tariff=t.id, description="blue")

pipr = ProductItemPartnerRef(ref_super=cpi.id)
piprr = ProductItemPartnerRefRevision(ref_partner=p.id, description="blue")

w1 = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w1)
create_component!(c, cr, w1)
create_subcomponent!(c, cpr, cprr, w1)
create_subcomponent!(c, cpi, cpir, w1)
create_subcomponent!(cpi, pitr, pitrr, w1)
create_subcomponent!(cpi, pipr, piprr, w1)
commit_workflow!(w1)


Update Contract Yellow

In [ ]:
# update Contract yellow

    cr1 = ContractRevision(ref_component = c.id, description = "yellow")
    cp2r = ContractPartnerRef(ref_super=c.id)
    cp2rr = ContractPartnerRefRevision(ref_partner=p2.id, description="blue")
    w2 = Workflow(
        ref_history = w1.ref_history,
        tsw_validfrom = ZonedDateTime(2016, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    update_entity!(w2)
    update_component!(cr, cr1, w2)
    create_subcomponent!(c, cp2r, cp2rr, w1)
    commit_workflow!(w2)
    @test w2.ref_history == w1.ref_history


Contract Update Red

In [8]:
cr2 = ContractRevision(ref_component = c.id, description = "red")
w3 = Workflow(
    ref_history = w2.ref_history,
    tsw_validfrom = ZonedDateTime(2015, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w3)
update_component!(cr1, cr2, w3)
commit_workflow!(w3)
@test w3.ref_history == w2.ref_history

# end of mutations

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/QUDiV/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'4' ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/QUDiV/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 6 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/QUDiV/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed" ) VALUES ( E'4', E'6', E'2038-01-19T03:14:06.999+00:00', E'2015-05-30T21:00:01.001+01:00', 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/QUDiV/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_hi

┌ Info: INSERT INTO validityintervals ( "ref_history", "ref_version", "tsdb_validfrom", "tsworld_validfrom", "tsdb_invalidfrom", "tsworld_invalidfrom", "is_committed" ) VALUES ( E'4', E'4', E'2022-04-01T18:15:00.472+01:00', E'2014-05-30T20:00:01.001+00:00', E'2038-01-19T03:14:06.999+00:00', E'2015-05-30T20:00:01.001+00:00', 1 ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/QUDiV/src/SearchLightPostgreSQL.jl:134
NOTICE:  NEW: (8,4,4,"2014-05-30 20:00:01.001+00","2015-05-30 20:00:01.001+00","2022-04-01 17:15:00.472+00","2038-01-19 03:14:06.999+00",,,1)
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidf

┌ Info: UPDATE workflows SET  "id" = E'6', "ref_history" = E'4', "ref_version" = E'6', "tsdb_validfrom" = E'2022-04-01T18:15:00.472+01:00', "tsw_validfrom" = E'2015-05-30T20:00:01.001+00:00', "is_committed" = 1 WHERE workflows.id = '6' RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/QUDiV/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_history" AS "workflows_ref_history", "workflows"."ref_version" AS "workflows_ref_version", "workflows"."tsdb_validfrom" AS "workflows_tsdb_validfrom", "workflows"."tsw_validfrom" AS "workflows_tsw_validfrom", "workflows"."is_committed" AS "workflows_is_committed" FROM "workflows" WHERE "id" = 6 ORDER BY workflows.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/QUDiV/src/SearchLightPostgreSQL.jl:134
┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/QUDiV/src/SearchLightPostgreSQL.jl:13

Test Passed